<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=694afb4f3b4c5f33bf9e1cd409ad557a5db1ab25a3c57aa614610646693067a3
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-18 10:21:35
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.39 C
-------------------
Today PnL: -49.19 K (-0.35%)
Current PnL: -28.25 L (-18.25%)
CY Booked + Current PnL: -13.96 L (-9.02%)
-------------------
Total profit:  1.28 L
Total loss:  -29.53 L
-------------------
Total Booked + Current PnL: 13.02 L (10.18%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.3%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.89 L (66.92%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.56%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.58,-19.63,24.55,0.09,19883.0,-19786.0,80991.0,88.05,29.0,M-SC,2.40,253.0,-1.00,0.59,6.40,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.92,8.99,9.82,19.69,20522.0,17238.0,208980.0,-1.63,57.0,X-MC,2.51,78.0,0.84,1.51,16.40,XY25,NTT,AC
50,MASFIN,398.61,-0.73,-4.24,27.45,22.05,25755.0,-4155.0,93825.0,-17.34,48.0,H-SC,6.77,164.0,-0.16,0.68,36.38,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.64,-16.28,19.46,0.02,28440.0,-28413.0,146146.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.06,27.12,OX40N,NTT,PAINTS
43,ITC,452.00,0.33,-1.69,12.69,10.78,29980.0,-4070.0,236248.0,-41.56,45.0,X-LC,2.05,5.0,-0.14,1.71,3.45,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.95,-12.09,106.67,81.67,87965.0,-11345.0,82465.0,7416.67,59.0,L-SC,12.42,271.0,-0.13,0.60,59.70,XR,NTT,CERAMICS
61,ROUTE,2227.21,2.28,-47.17,220.00,69.05,154651.0,-62770.0,70296.0,-57.12,52.0,H-SC,22.52,140.0,-0.41,0.51,8.41,SR,ATH,IT
13,BSOFT,831.70,1.53,-20.51,89.69,50.79,100278.0,-28848.0,111805.0,2.05,67.0,H-SC,6.64,151.0,-0.29,0.81,30.18,XR,ATH,IT
48,KPIGREEN,730.66,1.46,-14.91,72.71,46.95,77515.0,-18688.0,106609.0,-21.47,44.0,H-SC,8.43,154.0,-0.24,0.77,33.56,MH,ATH,POWER
41,INFY,1972.00,1.29,10.47,21.53,34.24,74415.0,32747.0,345635.0,-13.85,66.0,X-LC,4.56,2.0,0.44,2.50,19.62,X40,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.0,-3.09,1.43,19.06,20.77,45267.0,3354.0,237498.0,-7.46,38.0,X-MC,2.31,62.0,0.07,1.72,8.45,X40N,NTT,BREWERIES
56,RAJESHEXPO,518.0,-2.76,-63.24,172.24,0.07,86847.0,-86755.0,50422.0,1748.48,52.0,L-SC,2.37,268.0,-1.00,0.37,25.40,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.0,-2.27,-45.92,222.00,74.15,196437.0,-75120.0,88485.0,-259.08,40.0,H-SC,22.85,149.0,-0.38,0.64,13.42,XR,NTT,CHEMICALS
16,CERA,9475.0,-2.17,-30.53,78.34,23.89,95731.0,-53704.0,122199.0,-32.94,40.0,H-SC,11.05,157.0,-0.56,0.88,5.53,OX40N,NTT,CERAMICS
27,GLAXO,3466.2,-1.68,1.93,36.51,39.15,74165.0,3852.0,203136.0,-18.20,46.0,X-MC,10.53,60.0,0.05,1.47,31.50,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.48,-2.48,113.20,107.91,162986.0,-3663.0,143981.0,-20.55,55.0,M-SC,12.56,216.0,-0.02,1.04,7.35,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.64,-16.28,19.46,0.02,28440.0,-28413.0,146146.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.06,27.12,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.58,-19.63,24.55,0.09,19883.0,-19786.0,80991.0,88.05,29.0,M-SC,2.40,253.0,-1.00,0.59,6.40,OX40N,NTT,DURABLES
66,SIS,528.00,0.30,-23.98,59.52,21.26,50238.0,-26627.0,84405.0,1991.71,51.0,H-SC,4.95,166.0,-0.53,0.61,14.04,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.13,-25.98,53.30,13.47,106393.0,-70056.0,199611.0,-69.48,32.0,H-SC,6.05,158.0,-0.66,1.45,1.50,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.96,8.14,59.3,72.27,108660.0,13789.0,183237.0,-6.99,69.0,M-LC,2.8,99.0,0.13,1.33,14.82,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.96,8.14,59.30,72.27,108660.0,13789.0,183237.0,-6.99,69.0,M-LC,2.80,99.0,0.13,1.33,14.82,XR,NTT,IT
25,FINCABLES,1641.55,0.48,-2.48,113.20,107.91,162986.0,-3663.0,143981.0,-20.55,55.0,M-SC,12.56,216.0,-0.02,1.04,7.35,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.13,-3.74,114.76,106.72,136243.0,-4616.0,118720.0,-52.34,37.0,H-SC,3.44,139.0,-0.03,0.86,18.76,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.22,-3.40,40.40,35.63,81789.0,-7127.0,202447.0,-17.18,37.0,H-MC,2.76,119.0,-0.09,1.47,12.70,AR,ATH,PHARMA
37,IEX,219.00,-0.51,-5.32,56.98,48.63,107872.0,-10639.0,189315.0,-36.95,44.0,H-SC,13.55,136.0,-0.10,1.37,5.62,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.20,-19.14,92.27,55.47,181523.0,-46556.0,196730.0,-28.25,22.0,X-MC,13.95,64.0,-0.26,1.42,0.80,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.24,-10.70,43.47,28.11,78181.0,-21560.0,179850.0,-35.24,25.0,X-MC,13.45,55.0,-0.28,1.30,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.06,-24.39,103.01,53.50,235017.0,-73601.0,228150.0,-63.37,27.0,X-MC,3.79,58.0,-0.31,1.65,1.34,XY24,NTT,FMCG
75,TMPV,600.00,-1.50,-30.08,75.88,22.98,119854.0,-67946.0,157952.0,-27.96,27.0,X-LC,6.75,3.0,-0.57,1.14,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.38,-39.64,124.00,35.20,96302.0,-51007.0,77663.0,1.81,29.0,X-SC,16.46,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.35,-3.51,29.01,24.48,70498.0,-8844.0,243012.0,-81.54,38.0,X-SC,0.62,86.0,-0.13,1.76,18.81,X40N,NTT,MISC
12,BERGEPAINT,680.0,-0.96,-4.92,27.41,21.14,59246.0,-11190.0,216149.0,-11.64,36.0,X-MC,1.70,74.0,-0.19,1.56,21.83,XY24,NTT,PAINTS
20,DABUR,735.0,-0.38,-3.76,49.31,43.70,117723.0,-9327.0,238741.0,-11.62,33.0,X-MC,1.98,72.0,-0.08,1.73,10.72,XY24,BTT,FMCG
43,ITC,452.0,0.33,-1.69,12.69,10.78,29980.0,-4070.0,236248.0,-41.56,45.0,X-LC,2.05,5.0,-0.14,1.71,3.45,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.14,4.19,25.48,30.73,52298.0,8245.0,205250.0,-20.23,53.0,X-MC,2.30,75.0,0.16,1.49,17.49,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.06,-26.12,122.20,64.15,214815.0,-62161.0,175790.0,-56.75,31.0,X-SC,3.00,82.0,-0.29,1.27,0.0,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.38,-39.64,124.00,35.20,96302.0,-51007.0,77663.0,1.81,29.0,X-SC,16.46,92.0,-0.53,0.56,0.0,X40,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.48,-21.12,79.41,41.51,164958.0,-55635.0,207730.0,-6.08,33.0,X-MC,7.53,61.0,-0.34,1.50,0.0,XY25,ATH,FMCG
53,PAGEIND,51605.08,-0.24,-10.70,43.47,28.11,78181.0,-21560.0,179850.0,-35.24,25.0,X-MC,13.45,55.0,-0.28,1.30,0.0,X40,ATH,APPARELS
75,TMPV,600.00,-1.50,-30.08,75.88,22.98,119854.0,-67946.0,157952.0,-27.96,27.0,X-LC,6.75,3.0,-0.57,1.14,0.0,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.30,-31.83,109.54,42.84,48543.0,-20691.0,44315.0,-55.37,36.0,X-SC,35.21,83.0,-0.43,0.32,0.26,XY24,NTT,MISC
59,RELAXO,1176.00,-0.86,-48.48,195.63,52.31,146304.0,-70374.0,74786.0,-45.12,38.0,X-SC,7.28,91.0,-0.48,0.54,0.11,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.38,-39.64,124.00,35.20,96302.0,-51007.0,77663.0,1.81,29.0,X-SC,16.46,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.46,-5.98,31.97,24.08,38158.0,-7594.0,119356.0,-13.67,30.0,X-SC,5.42,89.0,-0.20,0.86,12.46,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.35,-17.19,73.06,43.32,98543.0,-27992.0,134880.0,-28.80,34.0,X-SC,5.01,90.0,-0.28,0.98,3.56,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.64,-10.96,35.56,20.71,119764.0,-41443.0,336794.0,-23.30,64.0,X-LC,4.07,1.0,-0.35,2.44,12.54,X40,ATH,IT
41,INFY,1972.00,1.29,10.47,21.53,34.24,74415.0,32747.0,345635.0,-13.85,66.0,X-LC,4.56,2.0,0.44,2.50,19.62,X40,NTT,IT
75,TMPV,600.00,-1.50,-30.08,75.88,22.98,119854.0,-67946.0,157952.0,-27.96,27.0,X-LC,6.75,3.0,-0.57,1.14,0.00,XY24,NTT,AUTO
81,VBL,671.64,-1.24,-6.07,44.44,35.67,131840.0,-19172.0,296670.0,-17.17,45.0,X-LC,3.88,4.0,-0.15,2.15,6.94,X40N,ATH,FMCG
43,ITC,452.00,0.33,-1.69,12.69,10.78,29980.0,-4070.0,236248.0,-41.56,45.0,X-LC,2.05,5.0,-0.14,1.71,3.45,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.37,-38.21,115.07,32.89,54895.0,-29500.0,47706.0,-688.88,50.0,L-MC,6.24,259.0,-0.54,0.35,31.38,XR,NTT,BANKS
6,ASIANTILES,137.00,2.95,-12.09,106.67,81.67,87965.0,-11345.0,82465.0,7416.67,59.0,L-SC,12.42,271.0,-0.13,0.60,59.70,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.73,-4.24,27.45,22.05,25755.0,-4155.0,93825.0,-17.34,48.0,H-SC,6.77,164.0,-0.16,0.68,36.38,XR,ATH,FINANCE
13,BSOFT,831.70,1.53,-20.51,89.69,50.79,100278.0,-28848.0,111805.0,2.05,67.0,H-SC,6.64,151.0,-0.29,0.81,30.18,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.64,-16.28,19.46,0.02,28440.0,-28413.0,146146.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.06,27.12,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.53,-20.51,89.69,50.79,100278.0,-28848.0,111805.0,2.05,67.0,H-SC,6.64,151.0,-0.29,0.81,30.18,XR,ATH,IT
84,WIPRO,420.00,0.96,8.14,59.30,72.27,108660.0,13789.0,183237.0,-6.99,69.0,M-LC,2.80,99.0,0.13,1.33,14.82,XR,NTT,IT
41,INFY,1972.00,1.29,10.47,21.53,34.24,74415.0,32747.0,345635.0,-13.85,66.0,X-LC,4.56,2.0,0.44,2.50,19.62,X40,NTT,IT
73,TCS,4389.97,0.64,-10.96,35.56,20.71,119764.0,-41443.0,336794.0,-23.30,64.0,X-LC,4.07,1.0,-0.35,2.44,12.54,X40,ATH,IT
32,HCLTECH,1922.01,0.36,7.79,15.72,24.73,40992.0,18837.0,260761.0,12.34,58.0,X-LC,6.86,7.0,0.46,1.89,24.21,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.39
1,25,44.88
2,50,76.67


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.83
MC    30.87
LC    24.29
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.64
X40      23.16
X40N     12.94
XR        9.82
XY25      9.11
AR        9.00
OX40N     7.55
SR        1.00
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.37
H-SC    23.76
X-LC    20.81
M-SC    11.62
X-SC     8.02
H-MC     4.82
L-SC     1.43
M-LC     1.33
M-MC     1.33
H-LC     1.12
L-LC     1.03
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.35,-8.91,43.92
IT,13.17,-15.53,73.62
FINANCE,9.44,-22.06,75.15
MISC,7.27,-29.28,82.17
ELECTRICAL,6.00,-13.36,54.88
PAINTS,5.84,-14.56,31.57
INSURANCE,4.80,-2.53,37.62
PHARMA,4.14,-2.79,35.14
AUTO,2.80,-35.16,82.02


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3239774.0,21
AR,1327706.0,10
XR,1307015.0,13
X40,1024853.0,14
X40N,897494.0,9
OX40N,749116.0,10
XY25,382634.0,6
SR,282784.0,2
MH,77515.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3693892.0,25
X-MC,1457315.0,16
M-SC,1453577.0,15
X-LC,852883.0,11
X-SC,810806.0,8
H-MC,414249.0,3
L-SC,264630.0,3
M-LC,108660.0,1
H-LC,78962.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1296578.0      6
           AR         903859.0      5
M-SC       XY24       831233.0      6
H-SC       XR         788230.0      7
X-MC       X40        505346.0      7
           XY24       411986.0      3
           X40N       354503.0      4
H-SC       OX40N      344926.0      4
X-LC       X40        325562.0      5
M-SC       OX40N      317343.0      5
X-SC       X40N       315345.0      3
           XY24       301516.0      3
H-SC       SR         282784.0      2
X-LC       X40N       227646.0      2
H-MC       AR         217331.0      2
X-LC       XY24       201543.0      2
H-MC       XY24       196918.0      1
X-SC       X40        193945.0      2
X-MC       XY25       185480.0      2
L-SC       XR         177783.0      2
M-SC       XR         177447.0      2
           AR         127554.0      2
M-LC       XR         108660.0      1
X-LC       XY25        98132.0      2
L-SC       OX40N       86847.0      1
H-LC       AR          78962.0      1
H-SC       MH          77515.0      1
M-MC       XY25        62061.0      1
L-MC       XR          54895.0      1
L-LC       XY25        36961.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
